This notebook adds even more features such as tonal, harmonic and tempo to the original features and saves them in a file called features_with_all_extracted.csv

In [10]:
import numpy as np

# Load data
X = np.load('X_Data.npy', allow_pickle=True)  
y = np.load('y_Genres.npy')  
file_names = np.load('file_names.npy')



In [12]:
import librosa
def extract_features(audio, sr=22050, n_mfcc=13):
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)

    # Zero-Crossing Rate
    zcr = np.mean(librosa.feature.zero_crossing_rate(audio))

    # RMS Energy
    rms = np.mean(librosa.feature.rms(y=audio))

    # Spectral Features
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=audio, sr=sr))
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sr), axis=1).flatten()
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sr))
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sr), axis=1).flatten()

    # Temporal Features
    tempo, _ = librosa.beat.beat_track(y=audio, sr=sr)

    # Harmonic-Percussive Features
    harmonic, percussive = librosa.effects.hpss(audio)
    harmonic_ratio = np.mean(harmonic) / (np.mean(harmonic) + np.mean(percussive))

    # Tonal Features
    tonnetz = np.mean(librosa.feature.tonnetz(y=audio, sr=sr), axis=1).flatten()

    # Spectral Flatness
    spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=audio))

    # Combine all features into a consistent 1D array
    features = np.concatenate([
        mfcc_mean, mfcc_std, [zcr, rms, spectral_centroid, spectral_bandwidth, spectral_rolloff, tempo, harmonic_ratio, spectral_flatness],
        spectral_contrast, chroma_stft, tonnetz
    ])
    return features





In [ ]:
data_rows = []


for i, audio in enumerate(X):
    try:
        features = extract_features(audio)
        data_rows.append(np.append(features, y[i]))
    except Exception as e:
        print(f"Error processing sample {i}: {e}")

mfcc_mean_cols = [f"mfcc_mean_{i}" for i in range(13)]
mfcc_std_cols = [f"mfcc_std_{i}" for i in range(13)]
spectral_contrast_cols = [f"spectral_contrast_{i}" for i in range(7)]  # Adjust for 7 bands
chroma_cols = [f"chroma_{i}" for i in range(12)]  # 12 pitch classes

columns = (
    ["file_name"] +
    ["label"] +
    mfcc_mean_cols +
    mfcc_std_cols +
    ["zcr", "rms", "spectral_centroid", "spectral_bandwidth", "spectral_rolloff"] +
    spectral_contrast_cols +
    chroma_cols
)
feature_df = pd.DataFrame(data_rows, columns=columns)

feature_df.insert(0, "file_name", file_names)
feature_df.to_csv("features_with_all_extracted.csv", index=False)
print("Features saved to features_with_all_extracted.csv")